In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from dash import Dash, dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import dash_dangerously_set_inner_html
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
mta_data = pd.read_csv('./data/MTA_Daily_Ridership.csv',parse_dates=['Date'])

In [ ]:
mta_data = mta_data.rename(columns={
            'Subways: Total Estimated Ridership' : 'Subways',
            'Subways: % of Comparable Pre-Pandemic Day' : 'Subways: % of Pre-Pandemic',
            'Buses: Total Estimated Ridership' : 'Buses',
            'Buses: % of Comparable Pre-Pandemic Day' : 'Buses: % of Pre-Pandemic',
            'LIRR: Total Estimated Ridership' : 'LIRR',
            'LIRR: % of Comparable Pre-Pandemic Day' : 'LIRR : % of Pre-Pandemic',
            'Metro-North: Total Estimated Ridership' : 'Metro-North',
            'Metro-North: % of Comparable Pre-Pandemic Day' : 'Metro-North: % of Pre-Pandemic',
            'Access-A-Ride: Total Scheduled Trips' : 'Access-A-Ride',
            'Access-A-Ride: % of Comparable Pre-Pandemic Day' : 'Access-A-Ride: % of Pre-Pandemic',
            'Bridges and Tunnels: Total Traffic' : 'Bridges and Tunnels',
            'Bridges and Tunnels: % of Comparable Pre-Pandemic Day' : 'Bridges and Tunnels: % of Pre-Pandemic',
            'Staten Island Railway: Total Estimated Ridership' : 'Staten Island Railway',
            'Staten Island Railway: % of Comparable Pre-Pandemic Day' : 'Staten Island Railway: % of Pre-Pandemic'
            },
            )

In [ ]:
mta_data_thousands = mta_data.copy()
columns_to_divide = [
    'Subways', 
    'Buses', 
    'LIRR', 
    'Metro-North',
    'Access-A-Ride',
    'Bridges and Tunnels',
    'Staten Island Railway'
]

# Perform the division and update only those columns
mta_data_thousands[columns_to_divide] = mta_data[columns_to_divide] / 1000

Here we create different datasets for different granualarities of date, daily is too low.
This will eventually be removed as I can change the granualarity dynamically depending on the user selection. 
So we only have 1 data frame instead of 4 this makes it more memory efficient

In [ ]:
monthly_data = mta_data_thousands.resample('ME', on='Date').mean() # 'ME' for monthly (ME = Month End)
monthly_data.reset_index(inplace=True)

In [ ]:
quarterly_data = mta_data_thousands.resample('QE', on='Date').mean()  # 'QE' for quarterly (QE = Quarter End)
quarterly_data.reset_index(inplace=True)

In [ ]:
weekly_data = mta_data_thousands.resample('W', on='Date').mean()  # 'W' for quarterly
weekly_data.reset_index(inplace=True)

In [ ]:
annual_data = mta_data_thousands.resample('YE', on='Date').mean()  # 'YE' for annual (YE = Year End)
annual_data.reset_index(inplace=True)
annual_data['Year'] = annual_data['Date'].dt.year.astype(str) # Convert to string for better looking axis labels

The following is The code for the Dash app I will keep it in one cell

In [ ]:
dbc_css = "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css"

app = Dash(
    __name__, external_stylesheets=[dbc.themes.PULSE, dbc_css]
) 

server = app.server 

app.layout = dbc.Container(
    children=[
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.H2(
                            id="report_title",
                            className="bg-primary text-white p-2 mb-2 text-center",
                        )
                    ]
                )
            ]
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dbc.Card(
                            [
                                